In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn import preprocessing, model_selection
from sklearn.metrics import log_loss
import string
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import  CountVectorizer
from scipy.stats import boxcox
from scipy import stats
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.feature_extraction.text import TfidfTransformer
from keras.regularizers import l2, activity_l2
from sklearn.neighbors import KNeighborsClassifier
from keras.callbacks import Callback
import xgboost as xgb

class NBatchLogger(Callback):
    def __init__(self):
        self.seen = 0

    def on_epoch_end(self, batch, logs={}):
        self.seen += 1
        if self.seen % 500 == 0:
            print('epoch {}: loss = {}, val = {}'.format(self.seen, logs['loss'], logs['val_loss'])) 
            
class FakeDMatrix:
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels
        self.num = len(data)

    def num_row(self):
        return self.num

    def slice(self, rindex):
        indices = np.zeros(self.num, dtype=np.bool)
        for index in rindex:
            indices[index] = True
        return FakeDMatrix(data=self.data[indices], labels=self.labels[indices])

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [2]:
train_X = pd.read_json('data/train_with_leaky_cv.json')
train_y = pd.read_json('data/train_interest.json')[0]
columns = train_X.columns
train_X_scaled = pd.DataFrame(StandardScaler().fit_transform(train_X), columns=columns)
#train_X_with_layer_1 = pd.read_json('data/train_scaled_with_leaky_and_added_features_cv.json')

# Add a bunch of features to train_X

In [3]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.neighbors import DistanceMetric

In [4]:
%%time
ntrain = int(len(train_X) * 0.75)

scaler = StandardScaler()

xtr = train_X[:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X[ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]


NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

dtrain = xgb.DMatrix(data=xtr, label=ytr)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

dtest = xgb.DMatrix(data=xte)
bst = xgb.train(params, dtrain, best_rounds)
xgb_preds = bst.predict(dtrain)
print('train score: ', log_loss(ytr_one_hot, xgb_preds))
xgb_preds = bst.predict(dtest)
print('test score: ', log_loss(yte_one_hot, xgb_preds))
# 0.539725

[0]	train-mlogloss:1.09155+1.6606e-05	test-mlogloss:1.09173+3.3003e-05
[200]	train-mlogloss:0.597445+0.00105445	test-mlogloss:0.628598+0.0028406
[400]	train-mlogloss:0.515812+0.000920072	test-mlogloss:0.570574+0.00291433
[600]	train-mlogloss:0.477461+0.000734069	test-mlogloss:0.553675+0.00268469
[800]	train-mlogloss:0.449402+0.00081771	test-mlogloss:0.545549+0.00230202
[1000]	train-mlogloss:0.426205+0.000555713	test-mlogloss:0.540721+0.00216
[1200]	train-mlogloss:0.406055+0.000566897	test-mlogloss:0.537628+0.00190077
[1400]	train-mlogloss:0.387419+0.000528787	test-mlogloss:0.535448+0.00163231
[1600]	train-mlogloss:0.370134+0.000625877	test-mlogloss:0.533992+0.00149583
[1800]	train-mlogloss:0.353876+0.000622638	test-mlogloss:0.533029+0.00138961
[2000]	train-mlogloss:0.338651+0.000582744	test-mlogloss:0.532491+0.00127657
[2200]	train-mlogloss:0.32426+0.000509889	test-mlogloss:0.532121+0.0013312
0.5319384
2323
train score:  0.342155768556
test score:  0.540635572492
CPU times: user 3h 50m

In [5]:
%%time
cols = [col for col in list(train_X.columns) if not col.startswith('sparse_')]
ntrain = int(len(train_X) * 0.75)

scaler = StandardScaler()

xtr = train_X[cols][:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X[cols][ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]


NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

dtrain = xgb.DMatrix(data=xtr, label=ytr)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

dtest = xgb.DMatrix(data=xte)
bst = xgb.train(params, dtrain, best_rounds)
xgb_preds = bst.predict(dtrain)
print('train score: ', log_loss(ytr_one_hot, xgb_preds))
xgb_preds = bst.predict(dtest)
print('test score: ', log_loss(yte_one_hot, xgb_preds))
# 0.539725

[0]	train-mlogloss:1.09158+1.2254e-05	test-mlogloss:1.09177+4.96411e-05
[200]	train-mlogloss:0.599951+0.00108291	test-mlogloss:0.631008+0.00339957
[400]	train-mlogloss:0.520519+0.0011692	test-mlogloss:0.575297+0.00369548
[600]	train-mlogloss:0.482412+0.00104068	test-mlogloss:0.559669+0.00376001
[800]	train-mlogloss:0.454044+0.000929429	test-mlogloss:0.552468+0.00344735
[1000]	train-mlogloss:0.430503+0.000806462	test-mlogloss:0.548359+0.00325123
[1200]	train-mlogloss:0.409636+0.000810443	test-mlogloss:0.545701+0.00313313
[1400]	train-mlogloss:0.390699+0.000842407	test-mlogloss:0.543955+0.00289192
[1600]	train-mlogloss:0.373082+0.000804968	test-mlogloss:0.542765+0.00285564
[1800]	train-mlogloss:0.356764+0.000801748	test-mlogloss:0.54211+0.00291224
[2000]	train-mlogloss:0.341275+0.000658387	test-mlogloss:0.541757+0.00292154
0.5417468
1983
train score:  0.367440078365
test score:  0.552152235208
CPU times: user 1h 2min 54s, sys: 7min 22s, total: 1h 10min 16s
Wall time: 25min 37s


In [6]:
%%time
cols = [col for col in list(train_X.columns) if col.startswith('sparse_')]
ntrain = int(len(train_X) * 0.75)

scaler = StandardScaler()

xtr = train_X[cols][:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X[cols][ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]


NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

dtrain = xgb.DMatrix(data=xtr, label=ytr)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

dtest = xgb.DMatrix(data=xte)
bst = xgb.train(params, dtrain, best_rounds)
xgb_preds = bst.predict(dtrain)
print('train score: ', log_loss(ytr_one_hot, xgb_preds))
xgb_preds = bst.predict(dtest)
print('test score: ', log_loss(yte_one_hot, xgb_preds))
# 0.539725

[0]	train-mlogloss:1.09348+1.86719e-05	test-mlogloss:1.09352+3.70373e-05
[200]	train-mlogloss:0.752481+0.00102072	test-mlogloss:0.760523+0.00329792
[400]	train-mlogloss:0.71743+0.00122986	test-mlogloss:0.731634+0.00417286
[600]	train-mlogloss:0.708357+0.00132663	test-mlogloss:0.726678+0.00448487
[800]	train-mlogloss:0.703372+0.0013688	test-mlogloss:0.72493+0.00462091
[1000]	train-mlogloss:0.69958+0.00135632	test-mlogloss:0.723996+0.00474291
[1200]	train-mlogloss:0.696258+0.00140188	test-mlogloss:0.723446+0.00477534
[1400]	train-mlogloss:0.6933+0.00141791	test-mlogloss:0.723054+0.004903
[1600]	train-mlogloss:0.690543+0.00142955	test-mlogloss:0.722823+0.00495664
[1800]	train-mlogloss:0.687899+0.00138574	test-mlogloss:0.722638+0.00505372
0.7226296
1867
train score:  0.691460796208
test score:  0.718937106058
CPU times: user 2h 14min 3s, sys: 5min 28s, total: 2h 19min 32s
Wall time: 43min 56s


In [7]:
%%time
cols = [col for col in list(train_X.columns) if (not col.startswith('worth_')) and (not col.startswith('price_'))]
ntrain = int(len(train_X) * 0.75)

scaler = StandardScaler()

xtr = train_X[cols][:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X[cols][ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]


NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

dtrain = xgb.DMatrix(data=xtr, label=ytr)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

dtest = xgb.DMatrix(data=xte)
bst = xgb.train(params, dtrain, best_rounds)
xgb_preds = bst.predict(dtrain)
print('train score: ', log_loss(ytr_one_hot, xgb_preds))
xgb_preds = bst.predict(dtest)
print('test score: ', log_loss(yte_one_hot, xgb_preds))
# 0.539725

[0]	train-mlogloss:1.09157+1.54065e-05	test-mlogloss:1.09175+4.85757e-05
[200]	train-mlogloss:0.603761+0.000821017	test-mlogloss:0.634796+0.00317821
[400]	train-mlogloss:0.522318+0.000672977	test-mlogloss:0.576766+0.0032528
[600]	train-mlogloss:0.482824+0.000676804	test-mlogloss:0.558888+0.00294804
[800]	train-mlogloss:0.454006+0.000947886	test-mlogloss:0.55008+0.00270453
[1000]	train-mlogloss:0.43037+0.000936597	test-mlogloss:0.544939+0.0027399
[1200]	train-mlogloss:0.409605+0.00101806	test-mlogloss:0.541477+0.00259963
[1400]	train-mlogloss:0.390706+0.000865508	test-mlogloss:0.539105+0.00255886
[1600]	train-mlogloss:0.373329+0.00101805	test-mlogloss:0.537478+0.00242666
[1800]	train-mlogloss:0.357109+0.000873868	test-mlogloss:0.536388+0.00228088
[2000]	train-mlogloss:0.341927+0.000778022	test-mlogloss:0.535801+0.00220229
[2200]	train-mlogloss:0.32751+0.000739542	test-mlogloss:0.535228+0.00218038
0.5350862
2344
train score:  0.344390289525
test score:  0.545005589891
CPU times: user 3h 

In [8]:
%%time
leaky = ['manager_worths',
 'building_worths',
 'address_worths',
 'boro_worths',
 'nta_worths',
 'manager_low_frac',
 'building_low_frac',
 'address_low_frac',
 'boro_low_frac',
 'nta_low_frac',
 'manager_med_frac',
 'building_med_frac',
 'address_med_frac',
 'boro_med_frac',
 'nta_med_frac',
 'manager_high_frac',
 'building_high_frac',
 'address_high_frac',
 'boro_high_frac',
 'nta_high_frac',
 'price_by_area',
 'real_minus_expected_price',
 'real_over_expected_price',
 'price_by_area',
 'real_minus_expected_price',
 'real_over_expected_price']
exclude = ['manager_worths',
 'building_worths',
 'address_worths',
 'boro_worths',
 'nta_worths',
 'price_by_area',
 'real_minus_expected_price',
 'real_over_expected_price',
 'price_by_area',
 'real_minus_expected_price',
 'real_over_expected_price']
cols = [col for col in list(train_X.columns) if not col in exclude]
ntrain = int(len(train_X) * 0.75)

scaler = StandardScaler()

xtr = train_X[cols][:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X[cols][ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]


NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

dtrain = xgb.DMatrix(data=xtr, label=ytr)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

dtest = xgb.DMatrix(data=xte)
bst = xgb.train(params, dtrain, best_rounds)
xgb_preds = bst.predict(dtrain)
print('train score: ', log_loss(ytr_one_hot, xgb_preds))
xgb_preds = bst.predict(dtest)
print('test score: ', log_loss(yte_one_hot, xgb_preds))
# 0.539725

[0]	train-mlogloss:1.09159+1.21425e-05	test-mlogloss:1.0918+2.84422e-05
[200]	train-mlogloss:0.59996+0.000772096	test-mlogloss:0.629595+0.00311747
[400]	train-mlogloss:0.519363+0.00077411	test-mlogloss:0.571818+0.00340136
[600]	train-mlogloss:0.481832+0.00100782	test-mlogloss:0.555029+0.00317221
[800]	train-mlogloss:0.454291+0.000841793	test-mlogloss:0.546885+0.00280976
[1000]	train-mlogloss:0.431295+0.000805227	test-mlogloss:0.542039+0.00258348
[1200]	train-mlogloss:0.411292+0.000437525	test-mlogloss:0.538816+0.00239402
[1400]	train-mlogloss:0.392856+0.000451856	test-mlogloss:0.536676+0.00228001
[1600]	train-mlogloss:0.375654+0.000453554	test-mlogloss:0.535165+0.00203467
[1800]	train-mlogloss:0.359837+0.000431358	test-mlogloss:0.534134+0.00194974
[2000]	train-mlogloss:0.344842+0.000421464	test-mlogloss:0.533491+0.00187691
[2200]	train-mlogloss:0.330511+0.000491137	test-mlogloss:0.533126+0.00189975
0.5330068
2347
train score:  0.346652655087
test score:  0.540746229861
CPU times: user 

In [9]:
%%time
leaky = ['manager_worths',
 'building_worths',
 'address_worths',
 'boro_worths',
 'nta_worths',
 'manager_low_frac',
 'building_low_frac',
 'address_low_frac',
 'boro_low_frac',
 'nta_low_frac',
 'manager_med_frac',
 'building_med_frac',
 'address_med_frac',
 'boro_med_frac',
 'nta_med_frac',
 'manager_high_frac',
 'building_high_frac',
 'address_high_frac',
 'boro_high_frac',
 'nta_high_frac',
 'price_by_area',
 'real_minus_expected_price',
 'real_over_expected_price',
 'price_by_area',
 'real_minus_expected_price',
 'real_over_expected_price']
exclude = ['manager_worths',
 'building_worths',
 'address_worths',
 'boro_worths',
 'nta_worths',]
cols = [col for col in list(train_X.columns) if not col in exclude]
ntrain = int(len(train_X) * 0.75)

scaler = StandardScaler()

xtr = train_X[cols][:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X[cols][ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]


NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

dtrain = xgb.DMatrix(data=xtr, label=ytr)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

dtest = xgb.DMatrix(data=xte)
bst = xgb.train(params, dtrain, best_rounds)
xgb_preds = bst.predict(dtrain)
print('train score: ', log_loss(ytr_one_hot, xgb_preds))
xgb_preds = bst.predict(dtest)
print('test score: ', log_loss(yte_one_hot, xgb_preds))
# 0.539725

[0]	train-mlogloss:1.09159+1.88425e-05	test-mlogloss:1.09178+3.87453e-05
[200]	train-mlogloss:0.598728+0.000835398	test-mlogloss:0.629345+0.00291743
[400]	train-mlogloss:0.517111+0.000724804	test-mlogloss:0.570925+0.00310297
[600]	train-mlogloss:0.478786+0.000630908	test-mlogloss:0.554056+0.00281684
[800]	train-mlogloss:0.450959+0.000847722	test-mlogloss:0.545784+0.00253017
[1000]	train-mlogloss:0.428243+0.000909388	test-mlogloss:0.540922+0.00226222
[1200]	train-mlogloss:0.408057+0.000964012	test-mlogloss:0.537721+0.00215893
[1400]	train-mlogloss:0.389692+0.000816921	test-mlogloss:0.535661+0.00209449
[1600]	train-mlogloss:0.372554+0.000741175	test-mlogloss:0.534111+0.00200406
[1800]	train-mlogloss:0.356591+0.000803446	test-mlogloss:0.533127+0.0019529
[2000]	train-mlogloss:0.341646+0.00086717	test-mlogloss:0.532504+0.00197845
[2200]	train-mlogloss:0.327331+0.000866273	test-mlogloss:0.532212+0.00200672
0.5321342
2253
train score:  0.34974735073
test score:  0.540574268734
CPU times: user

In [14]:
%%time
ntrain = int(len(train_X) * 0.75)

scaler = StandardScaler()

xtr = train_X[:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X[ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]


NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
}
params['objective'] = 'multi:softprob'
params['eta'] = 0.02
params['max_depth'] = 4
params['silent'] = 1
params['num_class'] = 3
params['eval_metric'] = "mlogloss"
params['min_child_weight'] = 1
params['subsample'] = 0.7
params['colsample_bytree'] = 0.7
params['seed'] = 321
params['nthread'] = 8

dtrain = xgb.DMatrix(data=xtr, label=ytr)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

dtest = xgb.DMatrix(data=xte)
bst = xgb.train(params, dtrain, best_rounds)
xgb_preds = bst.predict(dtrain)
print('train score: ', log_loss(ytr_one_hot, xgb_preds))
xgb_preds = bst.predict(dtest)
print('test score: ', log_loss(yte_one_hot, xgb_preds))
# 0.539725

[0]	train-mlogloss:1.08535+3.11294e-05	test-mlogloss:1.08548+0.000107357
[200]	train-mlogloss:0.570121+0.000726153	test-mlogloss:0.58647+0.00284809
[400]	train-mlogloss:0.528479+0.000697556	test-mlogloss:0.55939+0.00265658
[600]	train-mlogloss:0.505266+0.000626915	test-mlogloss:0.549434+0.00231266
[800]	train-mlogloss:0.487471+0.000620436	test-mlogloss:0.543988+0.00223315
[1000]	train-mlogloss:0.472287+0.00066562	test-mlogloss:0.540697+0.00195735
[1200]	train-mlogloss:0.458689+0.000742891	test-mlogloss:0.538394+0.00183402
[1400]	train-mlogloss:0.446194+0.000825688	test-mlogloss:0.536877+0.00171501
[1600]	train-mlogloss:0.434368+0.00080441	test-mlogloss:0.535797+0.00159622
[1800]	train-mlogloss:0.423102+0.00079824	test-mlogloss:0.534951+0.00149836
[2000]	train-mlogloss:0.412505+0.000778366	test-mlogloss:0.534443+0.00134322
[2200]	train-mlogloss:0.402362+0.000796937	test-mlogloss:0.534143+0.00136074
0.5341148
2177
train score:  0.422388568301
test score:  0.539069741444
CPU times: user 2

In [176]:
#%%time
#PCA
'''
n_components = 3
pca_columns = ['layer1_pca_{}'.format(i) for i in range(n_components)]
for column in pca_columns:
    train_X_with_layer_1[column] = 0
pca = PCA(n_components=n_components)
pca_representation = pca.fit_transform(train_X_scaled)
train_X_with_layer_1.loc[:, pca_cols] = pca_representation
'''

#tsne = TSNE(n_components = 2)
#tsne_representation = tsne.fit_transform(train_X_scaled[knn_features[:15]][:5000])

folds = 5
full_length = len(train_X_scaled)
entries = int(full_length / folds)
full_index = train_X_scaled.index
fold_indices = []
fold_indices_keep = []
fold_indices_throw = []
for i in range(folds):
    print ('Fold {}'.format(i))
    indices = sorted(random.sample(list(full_index), entries))
    fold_indices.append(indices)
    full_index = full_index.drop(indices)

    full_index_keep = np.zeros((full_length), dtype=np.bool)
    full_index_throw = np.ones((full_length), dtype=np.bool)
    for j in fold_indices[i]:
        full_index_keep[j] = True
        full_index_throw[j] = False
    fold_indices_throw.append(full_index_throw)
    fold_indices_keep.append(full_index_keep)
    
    xtr = train_X_scaled[full_index_throw]
    ytr = train_y[full_index_throw]
    xte = train_X_scaled[full_index_keep]
    
    '''
    knn_features = ['bc_price', 'manager_worths', 'latitude', 'longitude', 'desc_words_length', 'Yday', 'building_worths', 'price_per_bedroom']
    knn_n_closest = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
    knn_columns = ['layer1_knn_{}_{}'.format(n_neighbors, category) for category in ['high', 'medium', 'low'] for n_neighbors in knn_n_closest]
    for column in knn_columns:
        train_X_with_layer_1[column] = 0
    for n_neighbors in knn_n_closest:
        #knn_dist = DistanceMetric.get_metric('mahalanobis', V=np.eye(len(knn_n_closest)))
        knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric='manhattan') # metric='manhattan'
        knn.fit(train_X_scaled[knn_features][full_index_throw], train_y[full_index_throw])
        knn_pred = knn.predict_proba(train_X_scaled[knn_features][full_index_keep])
        new_cols = ['layer1_knn_{}_{}'.format(n_neighbors, category) for category in ['high', 'medium', 'low']]
        train_X_with_layer_1.loc[indices, new_cols] = knn_pred
    
    ridge_columns = ['layer1_ridge_predictions']
    for column in ridge_columns:
        train_X_with_layer_1[column] = 0
    ridge = Ridge()
    ridge.fit(xtr, ytr)
    ridge_predictions = ridge.predict(xte)
    train_X_with_layer_1.loc[indices, 'layer1_ridge_predictions'] = ridge_predictions
    
    rf_columns = ['layer1_rf_{}_{}_predictions'.format(criterion, category) for category in ['high', 'medium', 'low'] for criterion in ['gini', 'entropy']]
    for column in rf_columns:
        train_X_with_layer_1[column] = 0
    for criterion in ['gini', 'entropy']:
        rf = RandomForestClassifier(
            n_estimators=25,
            criterion=criterion,
            max_features='auto',
            min_samples_split=20)

        rf.fit(xtr, ytr)
        rf_predictions = rf.predict_proba(xte)
        new_cols = ['layer1_rf_{}_{}_predictions'.format(criterion, category) for category in ['high', 'medium', 'low']]
        train_X_with_layer_1.loc[indices, new_cols] = rf_predictions
    
    k_means_columns = ['layer1_k_means_{}_{}'.format(n_clusters, index) for index in range(n_clusters) for n_clusters in [2, 4, 8]]
    for column in k_means_columns:
        train_X_with_layer_1[column] = 0
    for n_clusters in [2, 4, 8]:
        k_means = KMeans(n_clusters=n_clusters)
        k_means.fit(xtr[knn_features], ytr)
        k_means_indices = k_means.predict(xte[knn_features])
        new_cols = ['layer1_k_means_{}_{}'.format(n_clusters, index) for index in range(n_clusters)]
        train_X_with_layer_1.loc[indices, new_cols] = np.eye(n_clusters)[k_means_indices]
    '''

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [180]:
train_X_with_layer_1.to_json('data/train_scaled_with_leaky_and_added_features_cv.json', orient='records')

In [276]:
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.layers.core import ActivityRegularization
def create_model(xtr, layer1=256, layer2=32, activation='tanh', activation2=PReLU, dropout=0.75):
    model = Sequential()
    
    model.add(Dense(layer1,
                    input_dim=xtr.shape[1],
                    init = 'he_normal',
                    activation=activation))
                    #kernel_regularizer=regularizers.l2(0.01),
                    #activity_regularizer=regularizers.activity_l2(0.005)))
    #model.add(Dense(64, input_dim=64, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
    
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    if activation2 != None:
        model.add(activation2())

    model.add(Dense(layer2,
                    init = 'he_normal',
                    activation=activation))
                    #activity_regularizer=regularizers.activity_l2(0.005)))
    model.add(BatchNormalization())    
    model.add(Dropout(dropout))
    if activation2 != None:
        model.add(activation2())

    model.add(Dense(3, init = 'he_normal', activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
    return model

In [268]:
def test_nn_with_columns(X, y, columns_to_include, layer1, layer2, activation, activation2, dropout):
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
    ntrain = int(len(X_scaled) * 0.75)

    xtr = X_scaled[columns_to_include][:ntrain]
    ytr = y[:ntrain].values
    ytr_one_hot = np.eye(3)[ytr]
    xte = X_scaled[columns_to_include][ntrain:]
    yte = y[ntrain:].values
    yte_one_hot = np.eye(3)[yte]

    filepath = 'data/nn_weights'

    logger = NBatchLogger()
    early_stop = EarlyStopping(monitor='val_loss', patience=40, verbose=0)
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)
    model = create_model(xtr, layer1, layer2, activation, activation2, dropout)
    model.fit(xtr.values, ytr_one_hot, nb_epoch = 1200, batch_size=1000, verbose = 0, validation_data=[xte.values, yte_one_hot], callbacks=[logger, early_stop, checkpoint])
    model = create_model(xtr, layer1, layer2, activation, activation2, dropout)
    model.load_weights(filepath)
    return log_loss(yte_one_hot, model.predict(xte.values))

# actually build model

In [274]:
ntrain = int(len(train_X_with_layer_1) * 0.75)

scaler = StandardScaler()

xtr = train_X_with_layer_1[:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X_with_layer_1[ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]

In [317]:
%%time
NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

xgb_cols = all_columns + k_means_columns + knn_columns + ridge_columns + pca_columns + rf_columns

params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

dtrain = xgb.DMatrix(data=xtr[xgb_cols], label=ytr)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

dtest = xgb.DMatrix(data=xte[xgb_cols])
bst = xgb.train(params, dtrain, best_rounds)
xgb_preds = bst.predict(dtest)
print(log_loss(yte_one_hot, xgb_preds))
# 0.539725

[0]	train-mlogloss:1.09111+1.37637e-05	test-mlogloss:1.09132+3.47597e-05
[200]	train-mlogloss:0.572365+0.000966518	test-mlogloss:0.606583+0.00289119
[400]	train-mlogloss:0.495784+0.00111757	test-mlogloss:0.556851+0.00274886
[600]	train-mlogloss:0.46142+0.00115805	test-mlogloss:0.546969+0.00247487
[800]	train-mlogloss:0.435288+0.00122307	test-mlogloss:0.543303+0.00232122
[1000]	train-mlogloss:0.412388+0.00110456	test-mlogloss:0.541574+0.00211891
[1200]	train-mlogloss:0.391678+0.0010726	test-mlogloss:0.540466+0.00189828
[1400]	train-mlogloss:0.372348+0.00112275	test-mlogloss:0.539871+0.00179687
0.5397984
1429
0.546284733463
CPU times: user 3h 44s, sys: 5min 4s, total: 3h 5min 48s
Wall time: 53min 48s


In [307]:
%%time
filepath = 'data/nn_weights'

nn_cols = all_columns + knn_columns + ridge_columns + k_means_columns

logger = NBatchLogger()
early_stop = EarlyStopping(monitor='val_loss', patience=40, verbose=0)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)
model = create_model(xtr[nn_cols])
model.fit(xtr[nn_cols].values, ytr_one_hot, nb_epoch = 1200, batch_size=1000, verbose = 0, validation_data=[xte[nn_cols].values, yte_one_hot], callbacks=[logger, early_stop, checkpoint])

model = create_model(xtr[nn_cols])
model.load_weights(filepath)
nn_preds = model.predict(xte[nn_cols].values)
print(log_loss(yte_one_hot, nn_preds))
# 0.58044096863

0.58044096863
CPU times: user 11min 51s, sys: 58.1 s, total: 12min 50s
Wall time: 10min 9s


In [310]:
%%time
et_cols = all_columns# + knn_columns + ridge_columns + k_means_columns

test = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=50)

test.fit(xtr[et_cols], ytr)
            
et_train_predictions = test.predict_proba(xtr[et_cols])
et_predictions = test.predict_proba(xte[et_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_predictions), log_loss(ytr_one_hot, et_train_predictions)))
# 0.5989926646715679

0.6084469079023 (test loss 0.5014715811903182)
CPU times: user 10 s, sys: 158 ms, total: 10.2 s
Wall time: 10.2 s


In [309]:
%%time
rf_cols = all_columns# + knn_columns + ridge_columns + k_means_columns

test = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=50)

test.fit(xtr[rf_cols], ytr)
            
rf_train_predictions = test.predict_proba(xtr[rf_cols])
rf_predictions = test.predict_proba(xte[rf_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_predictions), log_loss(ytr_one_hot, rf_train_predictions)))
# 0.5814357939419752

0.5867972591837729 (test loss 0.4744615099148531)
CPU times: user 11.2 s, sys: 297 ms, total: 11.5 s
Wall time: 11.6 s


In [304]:
%%time
test = KNeighborsClassifier(n_neighbors=500)
test.fit(xtr[non_sparse_cols], ytr)

knn_train_predictions = test.predict_proba(xtr[non_sparse_cols])
knn_predictions = test.predict_proba(xte[non_sparse_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, knn_predictions), log_loss(ytr_one_hot, knn_train_predictions)))

0.6644185840766779 (test loss 0.6542689234451893)
CPU times: user 5min 38s, sys: 4.82 s, total: 5min 43s
Wall time: 5min 55s


In [305]:
%%time
test = KNeighborsClassifier(n_neighbors=500)
test.fit(xtr[knn_features], ytr)

knn_train_predictions = test.predict_proba(xtr[knn_features])
knn_predictions = test.predict_proba(xte[knn_features])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, knn_predictions), log_loss(ytr_one_hot, knn_train_predictions)))

0.6542388412538083 (test loss 0.6351798773827041)
CPU times: user 49.6 s, sys: 1.34 s, total: 51 s
Wall time: 52.1 s


In [ ]:
print(log_loss(yte_one_hot, xgb_preds))
mixed_preds = (0.75 * xgb_preds) + (0.25 * nn_preds)
print(log_loss(yte_one_hot, mixed_preds))

In [ ]:
train thing on whether or not each classifier did a good job (minimize distance from correct?)
use result to go with the best prediction blended from models

In [45]:
#new_vects = ['tf_idf_' + x[7:] for x in sparse_cols]
#sparse = processed_train[sparse_cols]
#tf_idfs = TfidfTransformer().fit_transform(sparse)
#tf_idfs_df = pd.DataFrame(tf_idfs.toarray(), columns=new_vects)
#processed_train = processed_train.join(tf_idfs_df, how='left')

# Ridge Lasso Elastic

In [308]:
las = Lasso()
enet = ElasticNet()
las.fit(xte, yte)
enet.fit(xte, yte)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [309]:
print (las.score(xte, yte))
predictions = las.predict(xte)
print (log_loss(yte_one_hot, np.eye(3)[np.round(predictions).astype(int)]))

0.0
10.478087214


In [310]:
print (enet.score(xte, yte))
predictions = enet.predict(xte)
print (log_loss(yte_one_hot, np.eye(3)[np.round(predictions).astype(int)]))

0.0
10.478087214


In [42]:
for coef, column in zip(enet.coef_, train_X.columns):
    if coef != 0:
        print (column, ": ", coef)

In [43]:
for coef, column in zip(las.coef_, train_X.columns):
    if coef != 0:
        print (column, ": ", coef)

# TREES

In [322]:
test = ExtraTreesClassifier(
    n_estimators=100,
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
et_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_predictions), log_loss(ytr_one_hot, predictions)))

0.5843451108603969 (test loss 0.3912286072900344)


In [323]:
test = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
et_ent_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_ent_predictions), log_loss(ytr_one_hot, predictions)))

0.5841850290743527 (test loss 0.3945679429419886)


In [311]:
test = RandomForestClassifier(
    n_estimators=100,
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
_predictions = test.predict_proba(xtr)
_rf_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, _rf_predictions), log_loss(ytr_one_hot, _predictions)))

0.5702881670274706 (test loss 0.34260988988913094)


In [312]:
test = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
_predictions = test.predict_proba(xtr)
_rf_ent_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, _rf_ent_predictions), log_loss(ytr_one_hot, _predictions)))

0.5690015004443815 (test loss 0.34470505904504534)


In [337]:
pred_mixed = (0.01 * et_predictions + 0.01 * et_ent_predictions + 0.49 * rf_predictions + 0.49 * rf_ent_predictions)
log_loss(yte_one_hot, pred_mixed)

0.56688904952650898

In [12]:
%%time
test = KNeighborsClassifier(n_neighbors=2)
test.fit(xtr[non_sparse_cols], ytr)

predictions = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions = test.predict_proba(xte[non_sparse_cols])

7.076257047822155 (test loss 0.23129250627049086)
CPU times: user 3min 15s, sys: 1.48 s, total: 3min 16s
Wall time: 3min 20s


In [13]:
%%time
test = KNeighborsClassifier(n_neighbors=5)
test.fit(xtr[non_sparse_cols], ytr)

predictions_2 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_2 = test.predict_proba(xte[non_sparse_cols])

2.997161309463761 (test loss 0.4189513500335195)
CPU times: user 3min 36s, sys: 1.02 s, total: 3min 37s
Wall time: 3min 38s


In [14]:
%%time
test = KNeighborsClassifier(n_neighbors=10)
test.fit(xtr[non_sparse_cols], ytr)

predictions_3 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_3 = test.predict_proba(xte[non_sparse_cols])

1.5474359459916005 (test loss 0.4994428601539661)
CPU times: user 3min 48s, sys: 839 ms, total: 3min 49s
Wall time: 3min 50s


In [21]:
%%time
test = KNeighborsClassifier(n_neighbors=200)
test.fit(xtr[non_sparse_cols], ytr)

predictions_6 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_6 = test.predict_proba(xte[non_sparse_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_ent_predictions_6), log_loss(ytr_one_hot, predictions_6)))

0.6344317626130038 (test loss 0.6195293255389913)
CPU times: user 4min 34s, sys: 1.19 s, total: 4min 35s
Wall time: 4min 36s


In [ ]:
for feat, importance in zip(processed_train[cols].columns, clf.feature_importances_):
    if not np.isnan(importance) and importance > 0:
        print (feat, ": ", importance)

In [345]:
print ("nn: ", log_loss(yte_one_hot, nn_predictions))
print ("ab: ", log_loss(yte_one_hot, pred_mixed))
frac_nn = 0.2
averaged_preds = (nn_predictions * frac_nn) + (pred_mixed * (1 - frac_nn))
print ("combo: ", log_loss(yte_one_hot, averaged_preds))

nn:  0.563593691555
ab:  0.566889049527
combo:  0.55827286106


In [ ]:
train_X = pd.read_json('data/train_leaky.json')
train_y = pd.read_json('data/train_interest.json')[0]
scaler = StandardScaler().fit(train_X)
train_X = pd.DataFrame(scaler.transform(train_X), columns=columns)
test_X = pd.read_json('data/test_leaky.json')
test_X = pd.DataFrame(scaler.transform(test_X), columns=columns)
listing_id = pd.read_json('data/test_ids.json').values
preds = pd.read_csv('data/my_best_preds.csv')

In [295]:
preds = pd.read_csv('data/my_best_preds.csv')

In [313]:
et_clf = ExtraTreesClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='auto',
    min_samples_split=20)
et_clf.fit(train_X, train_y)          
et_predictions = et_clf.predict_proba(test_X)

In [314]:
et_clf_ent = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)
et_clf_ent.fit(train_X, train_y)          
et_ent_predictions = et_clf_ent.predict_proba(test_X)

In [316]:
rf_clf = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    min_samples_split=20)
rf_clf.fit(train_X, train_y)          
rf_predictions = rf_clf.predict_proba(test_X)

In [317]:
rf_clf_ent = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    min_samples_split=20)
rf_clf_ent.fit(train_X, train_y)          
rf_ent_predictions = rf_clf_ent.predict_proba(test_X)

In [259]:
nn_predictions = model.predict(test_X.values)

In [297]:
xgb_predictions = preds[['high', 'medium', 'low']].values

In [303]:
new_preds = (0.8 * xgb_predictions) + (0.15 * nn_predictions) + (0.05 * et_predictions)

In [304]:
new_preds_df = pd.DataFrame(new_preds, columns=['high', 'medium', 'low'])
new_preds_df['listing_id'] = preds['listing_id']

In [306]:
new_preds_df.to_csv('data/combined_preds.csv', index=None)